# World Model

## Imports


In [100]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler
import joblib
import os

## Load Dataset

In [101]:
def load_data(filepath="../dataset/dataset_limited_2.txt"):
    """Loads the dataset using pandas."""
    try:
        df = pd.read_csv(filepath)
        print(f"Dataset loaded successfully. Shape: {df.shape}")
        df = df.dropna()
        print(f"Shape after dropping NaNs: {df.shape}")
        return df
    except FileNotFoundError:
        print(f"Error: Dataset file not found at {filepath}")
        return None
    except Exception as e:
        print(f"Error loading dataset: {e}")
        return None

In [102]:
# 1. Load the dataset
dataframe = load_data()

Dataset loaded successfully. Shape: (1818, 14)
Shape after dropping NaNs: (1818, 14)


In [103]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1818 entries, 0 to 1817
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   distance_red_init     1818 non-null   float64
 1   angle_red_init        1818 non-null   float64
 2   distance_green_init   1818 non-null   float64
 3   angle_green_init      1818 non-null   float64
 4   distance_blue_init    1818 non-null   float64
 5   angle_blue_init       1818 non-null   float64
 6   rSpeed                1818 non-null   int64  
 7   lSpeed                1818 non-null   int64  
 8   distance_red_final    1818 non-null   float64
 9   angle_red_final       1818 non-null   float64
 10  distance_green_final  1818 non-null   float64
 11  angle_green_final     1818 non-null   float64
 12  distance_blue_final   1818 non-null   float64
 13  angle_blue_final      1818 non-null   float64
dtypes: float64(12), int64(2)
memory usage: 199.0 KB


## Preprocess Dataset

In [104]:
def prepare_data(df):
    """Separates features (X) and target variables (Y)."""
    # Input Features: initial state (6) + action (2) = 8 features
    X = df.iloc[:, :8].values
    # Target Variables: final state (6) = 6 features
    Y = df.iloc[:, 8:].values
    print(f"Features (X) shape: {X.shape}")
    print(f"Targets (Y) shape: {Y.shape}")
    return X, Y

In [105]:
def split_data(X, Y, test_size=0.2, random_state=42):
    """Splits data into training and testing sets."""
    X_train, X_test, Y_train, Y_test = train_test_split(
        X, Y, test_size=test_size, random_state=random_state
    )
    print(f"Training set size: {X_train.shape[0]} samples")
    print(f"Testing set size: {X_test.shape[0]} samples")
    return X_train, X_test, Y_train, Y_test

In [106]:
# 2. Prepare Data
X, Y = prepare_data(dataframe)

Features (X) shape: (1818, 8)
Targets (Y) shape: (1818, 6)


In [107]:
def scale_features(X_train, X_test):
    """Scales input features using StandardScaler."""
    
    scaler = StandardScaler()
    
    # Fit scaler ONLY on training data
    X_train_scaled = scaler.fit_transform(X_train)
    
    # Transform both train and test data
    X_test_scaled = scaler.transform(X_test)
    
    print("Features scaled.")
    
    return X_train_scaled, X_test_scaled, scaler # Return scaler to save it

In [108]:
# 3. Split Data
X_train, X_test, Y_train, Y_test = split_data(X, Y)

# 4. Scale Features (Important!)
X_train_scaled, X_test_scaled, scaler = scale_features(X_train, X_test)

Training set size: 1454 samples
Testing set size: 364 samples
Features scaled.


## Train model

In [109]:
# from sklearn.model_selection import GridSearchCV

# # Hyperparameter Tuning with GridSearchCV
# param_grid = {
#     'hidden_layer_sizes': [(64,), (32, 16), (64, 32), (100,)], # Try different depths/widths
#     'activation': ['relu', 'tanh'], # Common activation functions
#     'alpha': [0.0001, 0.001, 0.01], # Regularization strength
#     'learning_rate_init': [0.001, 0.0005, 0.005] # Initial learning rate
# }

# # Create the base MLP Regressor model instance for the grid search
# # Use slightly lower max_iter here to speed up the search process for each fold
# # Early stopping is still useful within each fold's training
# mlp_base = MLPRegressor(
#     solver='adam',
#     batch_size='auto',
#     learning_rate='constant',
#     max_iter=1500,
#     shuffle=True,
#     random_state=42,
#     tol=0.0001,
#     verbose=False,
#     warm_start=False,
#     early_stopping=True,
#     validation_fraction=0.1,
#     n_iter_no_change=15
# )

# # Set up GridSearchCV
# # cv=3 means 3-fold cross-validation
# # scoring='neg_mean_squared_error' - lower MSE is better, so we maximize the negative MSE.
# # n_jobs=-1 uses all available CPU cores for parallel processing.
# # verbose=2 shows more detailed progress.
# print("Starting GridSearchCV...")
# grid_search = GridSearchCV(
#     estimator=mlp_base,
#     param_grid=param_grid,
#     cv=5,
#     scoring='neg_root_mean_squared_error', # Optimize for lower RMSE (using negative)
#     n_jobs=-1,
#     verbose=1,
#     return_train_score=True # Optional: to see training scores as well
# )

# # Fit GridSearchCV on the SCALED training data
# # This will take some time!
# grid_search.fit(X_train_scaled, Y_train)

# # Print the best parameters and the corresponding score
# print("\nGridSearchCV Complete.")
# print(f"Best parameters found: {grid_search.best_params_}")
# # The score is negative RMSE, so closer to 0 is better. Convert back for interpretation.
# print(f"Best cross-validation score (RMSE): {-grid_search.best_score_:.4f}")

# # Get the best estimator found by the grid search
# best_mlp_model = grid_search.best_estimator_

# # print("\n--- Grid Search Results Summary ---")
# # You can optionally examine cv_results_ for more details
# pd.DataFrame(grid_search.cv_results_).sort_values(by='rank_test_score').head(10) # Show top 10 results

In [110]:
# from tqdm import tqdm

def train_model(X_train, Y_train):
    """Trains a regression model."""
    print("Training RandomForestRegressor model...")
    # Example using RandomForestRegressor
    # n_estimators: number of trees; random_state: reproducibility; n_jobs=-1: use all cores
    model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1, oob_score=True)

    model.fit(X_train, Y_train)
    print("Model training complete.")
    # For RandomForest, you can check the Out-of-Bag score as a quick estimate of performance
    if isinstance(model, RandomForestRegressor) and model.oob_score_:
         print(f"Model OOB score: {model.oob_score_:.4f}")
    return model

# import matplotlib.pyplot as plt

# def train_model(X_train, Y_train):
#     """Trains a regression model (now using MLPRegressor) with a progress bar and plots loss."""
#     print("Training MLPRegressor model...")
#     model = MLPRegressor(
#         hidden_layer_sizes=(64, 32),
#         activation='relu',
#         solver='adam',
#         alpha=0.0001,
#         batch_size='auto',
#         learning_rate='constant',
#         learning_rate_init=0.001,
#         max_iter=1000,
#         shuffle=True,
#         random_state=42,
#         tol=0.0001,
#         verbose=False,  # Disable built-in verbose to use custom progress bar
#         warm_start=False,
#         early_stopping=True,
#         validation_fraction=0.1,
#         n_iter_no_change=10
#     )

#     losses = []

#     # Custom training loop with tqdm progress bar
#     for i in tqdm(range(model.max_iter), desc="Training Progress"):
#         model.partial_fit(X_train, Y_train)
#         if hasattr(model, 'loss_'):
#             losses.append(model.loss_)
#         if len(losses) > 1 and abs(losses[-1] - losses[-2]) < model.tol:
#             print(f"Convergence reached at iteration {i + 1}")
#             break

#     print("Model training complete.")

#     # Plot the loss curve
#     plt.figure(figsize=(10, 6))
#     plt.plot(losses, label='Training Loss')
#     plt.xlabel('Iteration')
#     plt.ylabel('Loss')
#     plt.title('Training Loss Curve')
#     plt.legend()
#     plt.grid()
#     plt.show()

#     return model

In [ ]:
# # Replace your custom train_model function with this one
# import matplotlib.pyplot as plt
# from sklearn.neural_network import MLPRegressor  # Ensure this is imported

# def train_model(X_train, Y_train):
#     """Trains MLPRegressor using standard .fit() and plots training and validation loss."""
#     print("Training MLPRegressor model...")
#     model = MLPRegressor(
#         hidden_layer_sizes=(128, 64),  # Example architecture
#         activation='relu',
#         solver='adam',
#         alpha=0.0001,
#         batch_size='auto',
#         learning_rate='constant',
#         learning_rate_init=0.001,
#         max_iter=2000,  # Allow more iterations
#         shuffle=True,
#         random_state=42,
#         tol=0.0001,
#         verbose=False,
#         warm_start=False,
#         early_stopping=True,
#         validation_fraction=0.1,
#         n_iter_no_change=20
#     )

#     # Train the model
#     model.fit(X_train, Y_train)

#     # Plot the loss curves
#     if hasattr(model, 'loss_curve_') and hasattr(model, 'validation_scores_'):
#         fig, axes = plt.subplots(1, 2, figsize=(14, 6))

#         # Training loss subplot
#         axes[0].plot(model.loss_curve_, label='Training Loss', color='blue')
#         axes[0].set_xlabel('Iteration')
#         axes[0].set_ylabel('Loss')
#         axes[0].set_title('Training Loss Curve')
#         axes[0].legend()
#         axes[0].grid(True)

#         # Validation loss subplot
#         axes[1].plot(
#             range(len(model.validation_scores_)),
#             [-score for score in model.validation_scores_],
#             label='Validation Loss',
#             color='orange'
#         )
#         axes[1].set_xlabel('Iteration')
#         axes[1].set_ylabel('Loss')
#         axes[1].set_title('Validation Loss Curve')
#         axes[1].legend()
#         axes[1].grid(True)

#         plt.tight_layout()
#         plt.show()
#     else:
#         print("Could not plot loss curves (attributes not found).")

#     # Print additional information
#     if hasattr(model, 'n_iter_'):
#         print(f"Number of iterations run: {model.n_iter_}")
#     if hasattr(model, 'best_validation_score_'):
#         print(f"Best validation score achieved: {model.best_validation_score_:.4f}")

#     return model

In [111]:
# # 5. Train Model (using scaled data)
world_model = train_model(X_train_scaled, Y_train)

Training RandomForestRegressor model...
Model training complete.
Model OOB score: 0.9394
Model training complete.
Model OOB score: 0.9394


## Evaluate

In [112]:
def evaluate_model(model, X_test, Y_test):
    """Evaluates the model using MAE and MSE."""
    Y_pred = model.predict(X_test)

    mae = mean_absolute_error(Y_test, Y_pred)
    mse = mean_squared_error(Y_test, Y_pred)
    rmse = np.sqrt(mse) # Root Mean Squared Error

    print("\n--- Model Evaluation ---")
    print(f"Mean Absolute Error (MAE): {mae:.4f}")
    print(f"Mean Squared Error (MSE):  {mse:.4f}")
    print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")

    # Optional: Print metrics per output feature
    print("\nMAE per output feature:")
    output_features = [
        'dist_red_final', 'angle_red_final', 'dist_green_final',
        'angle_green_final', 'dist_blue_final', 'angle_blue_final'
    ]
    for i, name in enumerate(output_features):
         mae_feature = mean_absolute_error(Y_test[:, i], Y_pred[:, i])
         print(f"  {name}: {mae_feature:.4f}")

    return mae, mse

In [113]:
# # Cell [77] - Modify this cell

# # The grid search already trained the best model on the full training set
# # So, assign the best estimator directly to world_model
# print("Assigning best estimator from GridSearchCV to world_model...")
# world_model = best_mlp_model

# # Now, evaluate this best model on the held-out TEST set
# print("\nEvaluating the best model found by GridSearchCV on the TEST set:")
# # Assuming evaluate_model function is defined in cell [39]
# evaluate_model(world_model, X_test_scaled, Y_test)

# # Print the parameters of the best model found
# print("\nParameters of the best model:")
# print(world_model.get_params())

# # You can now proceed to save this 'world_model' (which is the best estimator)
# # using the save_model_and_scaler function in cell [23]

In [114]:
# 6. Evaluate Model (using scaled test data) New Data Random Forest
evaluate_model(world_model, X_test_scaled, Y_test)


--- Model Evaluation ---
Mean Absolute Error (MAE): 25.4712
Mean Squared Error (MSE):  1989.6590
Root Mean Squared Error (RMSE): 44.6056

MAE per output feature:
  dist_red_final: 41.5873
  angle_red_final: 10.6783
  dist_green_final: 37.0963
  angle_green_final: 18.5310
  dist_blue_final: 40.3401
  angle_blue_final: 4.5943


(25.471232276411712, 1989.6590256537377)

In [ ]:
# 6. Evaluate Model (using scaled test data) random forest
evaluate_model(world_model, X_test_scaled, Y_test)


--- Model Evaluation ---
Mean Absolute Error (MAE): 22.3253
Mean Squared Error (MSE):  1497.2351
Root Mean Squared Error (RMSE): 38.6941

MAE per output feature:
  dist_red_final: 39.9570
  angle_red_final: 3.9896
  dist_green_final: 38.2818
  angle_green_final: 7.6712
  dist_blue_final: 40.9996
  angle_blue_final: 3.0525


(22.325294401606275, 1497.2351473844744)

In [ ]:
# 6. Evaluate Model (using scaled test data) ANN
evaluate_model(world_model, X_test_scaled, Y_test)


--- Model Evaluation ---
Mean Absolute Error (MAE): 26.0102
Mean Squared Error (MSE):  1764.5742
Root Mean Squared Error (RMSE): 42.0068

MAE per output feature:
  dist_red_final: 40.2118
  angle_red_final: 7.6596
  dist_green_final: 42.1884
  angle_green_final: 14.1142
  dist_blue_final: 45.2772
  angle_blue_final: 6.6099


(26.01019102596153, 1764.5742434315123)

## Save model

In [116]:
def save_model_and_scaler(model, scaler, model_filename="world_model.joblib", scaler_filename="scaler.joblib"):
    """Saves the trained model and scaler to disk."""
    try:
        # Ensure the directory exists
        model_dir = "../src/models"
        os.makedirs(model_dir, exist_ok=True)

        model_path = os.path.join(model_dir, model_filename)
        scaler_path = os.path.join(model_dir, scaler_filename)

        joblib.dump(model, model_path)
        joblib.dump(scaler, scaler_path)
        print(f"Model saved to {model_path}")
        print(f"Scaler saved to {scaler_path}")
    except Exception as e:
        print(f"Error saving model/scaler: {e}")

In [117]:
# 7. Save Model and Scaler
save_model_and_scaler(world_model, scaler)

Model saved to ../src/models/world_model.joblib
Scaler saved to ../src/models/scaler.joblib


In [115]:
# Example prediction (how you'd use it later)
print("\n--- Example Prediction ---")
# Take the first sample from the original test set
sample_X = X_test[0].reshape(1, -1)
sample_Y_actual = Y_test[0]
# Scale the sample using the *saved* scaler
sample_X_scaled = scaler.transform(sample_X)
# Predict using the trained model
sample_Y_pred = world_model.predict(sample_X_scaled)

print(f"Input State + Action: {sample_X[0]}")
print(f"Actual Final State:   {sample_Y_actual}")
print(f"Predicted Final State:{sample_Y_pred[0]}")


--- Example Prediction ---
Input State + Action: [ 4.10674455e+02  1.22678632e+02  1.39104272e+03 -1.74541181e+02
  8.04670468e+02  5.43256407e+01  1.00000000e+00 -6.00000000e+00]
Actual Final State:   [ 419.68603568  120.04972943 1377.94257775 -175.22419136  825.72434115
   54.38562828]
Predicted Final State:[ 416.43578102  126.10140724 1383.72109556 -173.69369458  807.56981469
   55.83563045]
